In [1]:
from llama_index.core import StorageContext, load_index_from_storage
from constants import embedding_model

storage_context = StorageContext.from_defaults(persist_dir="index/")
index = load_index_from_storage(storage_context, embedding_model=embedding_model)

In [2]:
from llama_index.core.tools import QueryEngineTool
from constants import llm_model

query_engine = index.as_query_engine(llm_model=llm_model, similarity_top_k=5)
rag_tool = QueryEngineTool.from_defaults(
    query_engine, 
    name="reesarch_paper_query_engine_tool", 
    description="A RAG engine with recent research papers."
    )


In [3]:
from IPython.display import display, Markdown

def display_prompt_dictionary(prompt_dictionary):
    for key, prompt in prompt_dictionary.items():
        display(Markdown(f"**Prompt key**:{key}"))
        print(prompt.get_template())

In [4]:
prompts_dictionary = query_engine.get_prompts()
display_prompt_dictionary(prompts_dictionary)

**Prompt key**:response_synthesizer:text_qa_template

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt key**:response_synthesizer:refine_template

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [5]:
from tools import download_pdf, fetch_from_arxiv
from llama_index.core.tools import FunctionTool

download_pdf_tool = FunctionTool.from_defaults(
    download_pdf, 
    name="download_pdf_file_tool", 
    description="Python function that downloads PDFs from provided URLs."
    )

fetch_arxiv_tool = FunctionTool.from_defaults(
    fetch_from_arxiv, 
    name="fetch_from_arxiv", 
    description="Download the {max_results} recent papers regarding the topic {title} from arXiv."
)

In [6]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [rag_tool, download_pdf_tool, fetch_arxiv_tool], 
    llm_model=llm_model, 
    verbose=True
    )

In [7]:
query_template = """I am intersted in {topic}.
Find papers in your knowledge base that discuss this topic.
Use the folling template to query research_paper_query_engine_tool tool: 
'Provide title, summary, author and link to download the PDF for papers related to {topic}'
If there are no papers, could you fetch the recent ones from arXiv?
"""

In [8]:
answer = agent.chat(query_template.format(topic="Large Language Models"))

> Running step fea4bc67-149c-4f6d-9e04-9c14b2a68206. Step input: I am intersted in Large Language Models.
Find papers in your knowledge base that discuss this topic.
Use the folling template to query research_paper_query_engine_tool tool: 
'Provide title, summary, author and link to download the PDF for papers related to Large Language Models'
If there are no papers, could you fetch the recent ones from arXiv?

Thought: The user is interested in finding research papers related to Large Language Models. I can use the research_paper_query_engine_tool to search for papers on this topic. If there are no relevant papers in the knowledge base, I can fetch recent ones from arXiv.
Action: reesarch_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, author and link to download the PDF for papers related to Large Language Models'}
Observation: Title: How do Large Language Models Understand Relevance? A Mechanistic Interpretability Perspective
Summary: Recent studies have sho

In [9]:
Markdown(answer.response)

1. Title: How do Large Language Models Understand Relevance? A Mechanistic Interpretability Perspective
   Summary: This paper explores how different modules of Large Language Models (LLMs) contribute to relevance judgment through mechanistic interpretability.
   Authors: Qi Liu, Jiaxin Mao, Ji-Rong Wen
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07898v1)

2. Title: Porting an LLM based Application from ChatGPT to an On-Premise Environment
   Summary: The paper discusses the process of porting a real-life application using ChatGPT from a public cloud to an on-premise environment.
   Authors: Teemu Paloniemi, Manu Setälä, Tommi Mikkonen
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07907v1)

3. Title: Redefining Machine Translation on Social Network Services with Large Language Models
   Summary: This paper introduces RedTrans, a Large Language Model tailored for Social Network Services translation, outperforming state-of-the-art models.
   Authors: Hongcheng Guo, Fei Zhao, Shaosheng Cao, Xinze Lyu, Ziyan Liu, Yue Wang, Boyang Wang, Zhoujun Li, Chonggang Lu, Zhe Xu, Yao Hu
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07901v1)

4. Title: Cat, Rat, Meow: On the Alignment of Language Model and Human Term-Similarity Judgments
   Summary: The paper evaluates language models for their alignment with human similarity judgments on a word triplet task.
   Authors: Lorenz Linhardt, Tom Neuhäuser, Lenka Tětková, Oliver Eberle
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07965v1)

5. Title: GLUS: Global-Local Reasoning Unified into A Single Large Language Model for Video Segmentation
   Summary: This paper proposes a framework using multi-modal large language models for video object segmentation, achieving state-of-the-art results.
   Authors: Lang Lin, Xueyang Yu, Ziqi Pang, Yu-Xiong Wang
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07962v1)

In [10]:
answer = agent.chat("Downlod all the papers you just metioned.")

> Running step 2d6e6277-778a-4601-a1c4-421ac3db1c73. Step input: Downlod all the papers you just metioned.
Thought: I can download the PDF files of the papers mentioned using the download_pdf_file_tool.
Action: download_pdf_file_tool
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2504.07898v1', 'output_file_name': 'LLM_Understand_Relevance.pdf'}
Observation: PDF saved as 'papers\LLM_Understand_Relevance.pdf'.
> Running step ed67e6e6-aed4-4042-b060-8f3baabdec25. Step input: None
Thought: I can download the PDF files of the remaining papers mentioned using the download_pdf_file_tool.
Action: download_pdf_file_tool
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2504.07907v1', 'output_file_name': 'LLM_Porting_Application.pdf'}
Observation: PDF saved as 'papers\LLM_Porting_Application.pdf'.
> Running step 9bb14c03-4040-44c4-a5e6-4896f6494766. Step input: None
Thought: I can download the PDF files of the remaining papers mentioned using the download_pdf_file_tool.
Action: download_pdf_file

ValueError: Reached max iterations.

In [11]:
Markdown(answer.response)

1. Title: How do Large Language Models Understand Relevance? A Mechanistic Interpretability Perspective
   Summary: This paper explores how different modules of Large Language Models (LLMs) contribute to relevance judgment through mechanistic interpretability.
   Authors: Qi Liu, Jiaxin Mao, Ji-Rong Wen
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07898v1)

2. Title: Porting an LLM based Application from ChatGPT to an On-Premise Environment
   Summary: The paper discusses the process of porting a real-life application using ChatGPT from a public cloud to an on-premise environment.
   Authors: Teemu Paloniemi, Manu Setälä, Tommi Mikkonen
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07907v1)

3. Title: Redefining Machine Translation on Social Network Services with Large Language Models
   Summary: This paper introduces RedTrans, a Large Language Model tailored for Social Network Services translation, outperforming state-of-the-art models.
   Authors: Hongcheng Guo, Fei Zhao, Shaosheng Cao, Xinze Lyu, Ziyan Liu, Yue Wang, Boyang Wang, Zhoujun Li, Chonggang Lu, Zhe Xu, Yao Hu
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07901v1)

4. Title: Cat, Rat, Meow: On the Alignment of Language Model and Human Term-Similarity Judgments
   Summary: The paper evaluates language models for their alignment with human similarity judgments on a word triplet task.
   Authors: Lorenz Linhardt, Tom Neuhäuser, Lenka Tětková, Oliver Eberle
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07965v1)

5. Title: GLUS: Global-Local Reasoning Unified into A Single Large Language Model for Video Segmentation
   Summary: This paper proposes a framework using multi-modal large language models for video object segmentation, achieving state-of-the-art results.
   Authors: Lang Lin, Xueyang Yu, Ziqi Pang, Yu-Xiong Wang
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07962v1)

In [12]:
answer = agent.chat(query_template.format(topic="Quntum Computing"))

> Running step 3b94512d-5107-4266-b05e-683ae996eb2a. Step input: I am intersted in Quntum Computing.
Find papers in your knowledge base that discuss this topic.
Use the folling template to query research_paper_query_engine_tool tool: 
'Provide title, summary, author and link to download the PDF for papers related to Quntum Computing'
If there are no papers, could you fetch the recent ones from arXiv?

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: reesarch_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, author and link to download the PDF for papers related to Quantum Computing'}
Observation: Title: None
Summary: None
Authors: None
PDF URL: None
> Running step 80a8a651-93ce-45af-9eac-05f9b5abe4f1. Step input: None
Thought: The tool did not provide any papers related to Quantum Computing in the knowledge base.
Action: fetch_from_arxiv
Action Input: {'title': 'Quantum Computing', 'paper_count': 

In [13]:
Markdown(answer.response)

1. Title: Quantum Speed Limit in Driven-dissipative Systems
   Summary: This work investigates how drive-induced dissipation affects quantum speed limits in open quantum systems, enabling robust quantum control under realistic dissipative conditions.
   Authors: Sarfraj Fency, Riddhi Chatterjee, Rangeet Bhattacharyya
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07931v1)

2. Title: Temporal Tensors and Quantum Shortcut Dynamics in a Supermaze of Multidimensional Time
   Summary: A theoretical framework is developed to explore novel phenomena in quantum and classical systems using multiple time dimensions, quantum shortcut dynamics, and complex topological structures.
   Authors: Koffka Khan
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07900v1)

3. Title: QubitHammer Attacks: Qubit Flipping Attacks in Multi-tenant Superconducting Quantum Computers
   Summary: This work introduces QubitHammer attacks targeting superconducting quantum computers in multi-tenant cloud environments, demonstrating potential security vulnerabilities.
   Authors: Yizhuo Tan, Navnil Choudhury, Kanad Basu, Jakub Szefer
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07875v1)

4. Title: A quantum computing approach to beam angle optimization
   Summary: QC-BAO, a hybrid quantum-classical approach, is proposed for beam angle optimization in radiation therapy treatment planning, showcasing improved treatment plan quality.
   Authors: Nimita Shinde, Ya-Nan Zhu, Haozheng Shen, Hao Gao
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07844v1)

5. Title: Efficient Formal Verification of Quantum Error Correcting Programs
   Summary: An efficient verification framework for quantum error correction programs is presented, including a sound proof system and an automated verifier, Veri-QEC.
   Authors: Qifan Huang, Li Zhou, Wang Fang, Mengyu Zhao, Mingsheng Ying
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07732v1)